In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from PIL import Image
import tensorflow as tf
import os
import pandas as pd
import numpy

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def randfloat(a, b):
    return rand.random() * (b - a) + a

def my_print(s):
    sys.stdout.write(s)
    sys.stdout.flush()

In [4]:
data_dir = "./data/arrows/"
save_dir = "./save_col"
save_file_path = os.path.join(save_dir, "model.ckpt")
logs_dir = "./logs_col"
logs_dir_2 = "./logs2_col"

create_dir(save_dir)
create_dir(logs_dir)
create_dir(logs_dir_2)

BATCH_SIZE = 64
IMAGE_SIZE = 128

In [5]:
def _create_label_producers():
    labels_fname = os.path.join(data_dir, "labels_col.csv")
    labels = pd.read_csv(labels_fname, header=None)
    lines = [os.path.join(data_dir, "input_col", str(l[0]) + ".png") + " " + 
             str((l[1] + 1) / 2) for l in labels.values]
#     lines = lines[:10]
#     print lines
    cnt = len(lines)
    p = 0.9
    train_cnt = int(cnt * p)
    my_print("Total: %i images, train: %i\n" % (cnt, train_cnt))
    train_lines = lines[:train_cnt]
    test_lines = lines[train_cnt:]
    return tf.train.string_input_producer(train_lines), tf.train.string_input_producer(test_lines)

def _read_image_and_label(path_label_line):
    with tf.name_scope("read_image_and_label"):
        path, label_str = tf.decode_csv(path_label_line, [[""], [""]], field_delim=" ")
        label = tf.cast(tf.string_to_number(label_str, out_type=tf.int32), tf.int64)
        file_content = tf.read_file(path)
        image3c = tf.cast(tf.image.decode_png(file_content, channels=3, dtype=tf.uint8), tf.float32)
        image3c_w = image3c
        #image3c_w = tf.image.per_image_whitening(image3c)
        image1, image2 = tf.split(1, 2, image3c_w)
        image1.set_shape((IMAGE_SIZE, IMAGE_SIZE, 3))
        image2.set_shape((IMAGE_SIZE, IMAGE_SIZE, 3))
    return image1, image2, label

def _generate_batch(image1, image2, label, batch_size, min_after_dequeue=5000):
    with tf.name_scope("generate_batch"):
        images1, images2, labels = tf.train.shuffle_batch([image1, image2, label], batch_size=batch_size, 
                                                capacity=min_after_dequeue + batch_size * 4, 
                                                min_after_dequeue=min_after_dequeue,
                                                num_threads=5)
    return images1, images2, tf.reshape(labels, [batch_size])
    
def get_data_batch():
    train_queue, test_queue = _create_label_producers()
    image1_train, image2_train, label_train = _read_image_and_label(train_queue.dequeue())
    image1_test, image2_test, label_test = _read_image_and_label(test_queue.dequeue())
    images1_train, images2_train, labels_train = _generate_batch(image1_train, image2_train, 
                                                                 label_train, BATCH_SIZE)
    images1_test, images2_test, labels_test = _generate_batch(image1_test, image2_test, 
                                                              label_test, BATCH_SIZE,
                                                              min_after_dequeue=BATCH_SIZE)
    return images1_train, images2_train, labels_train, images1_test, images2_test, labels_test

def dense_to_one_hot(label_batch, num_labels=2):
    with tf.name_scope("one_hot_encoder"):
        sparse_labels = tf.cast(tf.reshape(label_batch, [-1, 1]), tf.int32)
        derived_size = tf.shape(sparse_labels)[0]
        indices = tf.reshape(tf.range(0, derived_size, 1), [-1, 1])
        concated = tf.concat(1, [indices, sparse_labels])
        outshape = tf.concat(0, [tf.reshape(derived_size, [1]), tf.reshape(num_labels, [1])])
        labels = tf.sparse_to_dense(concated, outshape, 1.0, 0.0)
        labels = tf.cast(labels, tf.float32)
    return labels

In [6]:
def _activation_summary(x):
    tensor_name = x.op.name
    tf.histogram_summary(tensor_name + '/activations', x)
    tf.scalar_summary(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


def _create_conv(layer, kernel_shape, sd=1e-3, bias=0.0, 
                 activation=tf.nn.relu, name="conv", trainable=True):
    with tf.name_scope(name):
        kernel = tf.Variable(tf.truncated_normal(kernel_shape, stddev=sd), name="kernel", 
                             trainable=trainable)
        tf.add_to_collection('main', kernel)
        conv = tf.nn.conv2d(layer, kernel, strides=[1, 1, 1, 1], padding="SAME")
#         biases = tf.constant(bias, shape=[kernel_shape[3]], name="biases")
        biases = tf.Variable(tf.constant(bias, shape=[kernel_shape[3]]), name="biases", trainable=trainable)
        tf.add_to_collection('main', biases)
        if activation:
            biased = tf.nn.bias_add(conv, biases)
            result = activation(biased)
        else:
            result = tf.nn.bias_add(conv, biases)
        _activation_summary(result)

    return result
    
def _create_pool(layer, kernel_width, stride=1, name="pool"):
    with tf.name_scope(name):
        pool = tf.nn.max_pool(layer, ksize=[1, kernel_width, kernel_width, 1], strides=[1, stride, stride, 1],
                             padding='SAME')
    return pool

def _create_norm(layer, name="norm"):
    with tf.name_scope(name):
#         norm = tf.nn.lrn(layer, alpha=0.001 / 9.0, beta=0.75)
        norm = tf.nn.lrn(layer)
    return norm

def _create_full(layer, result_size, weights_stddev=.04, activation=tf.nn.relu, name="full", trainable=True):
    with tf.name_scope(name):
        layer_size = 1
        for k in layer.get_shape()[1:].as_list():
            layer_size *= k
        reshaped = tf.reshape(layer, [BATCH_SIZE, layer_size])
        weights = tf.Variable(tf.truncated_normal([layer_size, result_size], stddev=weights_stddev), name="weights",
                             trainable=trainable)
        tf.add_to_collection('main', weights)
        biases = tf.Variable(tf.constant(.1, shape=[result_size]), name="biases", trainable=trainable)
        tf.add_to_collection('main', biases)
        if activation:
            biased = tf.add(tf.matmul(reshaped, weights), biases)
            result = activation(biased)
        else:
            result = tf.add(tf.matmul(reshaped, weights), biases)
        
        _activation_summary(result)
    return result
    
def build_classifier(images1, images2, trainable=True):
    images = tf.concat(3, [images1, images2])
    
    conv1 = _create_conv(images, [5, 5, 6, 64], bias=0.0, sd=1e-4, activation=tf.nn.tanh, 
                         name="conv1", trainable=trainable)
    pool1 = _create_pool(conv1, 3, 2, name="pool1")
    
    conv2 = _create_conv(pool1, [5, 5, 64, 128], bias=0.0, sd=1e-4, activation=tf.nn.tanh, 
                         name="conv2", trainable=trainable)
    pool2 = _create_pool(conv2, 3, 2, name="pool2")
    
    conv3 = _create_conv(pool2, [5, 5, 128, 128], bias=0.0, sd=1e-4, activation=tf.nn.tanh, 
                         name="conv3", trainable=trainable)
    pool3 = _create_pool(conv3, 3, 2, name="pool3")
    
    conv4 = _create_conv(pool3, [5, 5, 128, 128], bias=0.0, sd=1e-4, activation=tf.nn.tanh, 
                         name="conv4", trainable=trainable)
    pool4 = _create_pool(conv4, 3, 2, name="pool4")
    
    conv5 = _create_conv(pool4, [3, 3, 128, 2], bias=0.0, sd=1e-4, activation=tf.nn.tanh, 
                         name="conv5", trainable=trainable)
#     pool5 = _create_pool(conv5, 3, 2, name="pool5")
    
#     full1 = _create_full(pool3, 2, name="full1", activation=None, trainable=trainable)
#     softmax = tf.nn.softmax(full1, name="softmax")
    
#     conv_fin = _create_conv(pool4, [3, 3, 16, 2], bias=0.0, sd=1e-2, 
#                          activation=tf.nn.tanh, name="conv4", trainable=trainable)
    reduced = tf.reduce_mean(conv5, reduction_indices=[1, 2], name="reduce")

    softmax = tf.nn.softmax(reduced, name="softmax")

#     images = tf.concat(3, [images1, images2])
    
#     conv1 = _create_conv(images, [5, 5, 6, 16], name="conv1", trainable=trainable)
#     pool1 = _create_pool(conv1, 3, 2, name="pool1")
#     norm1 = _create_norm(pool1, name="norm1")
    
#     conv2 = _create_conv(norm1, [5, 5, 16, 32], name="conv2", trainable=trainable)
#     pool2 = _create_pool(conv2, 3, 2, name="pool2")
#     norm2 = _create_norm(pool2, name="norm2")
    
#     conv3 = _create_conv(norm2, [4, 4, 32, 32], name="conv3", trainable=trainable)
#     pool3 = _create_pool(conv3, 3, 2, name="pool3")
# #     norm3 = _create_norm(pool3, name="norm3")
    
#     full1 = _create_full(pool3, 256, name="full1", activation=tf.nn.relu, trainable=trainable)
#     full2 = _create_full(full1, 128, name="full2", activation=tf.nn.relu, trainable=trainable)
#     full3 = _create_full(full2, 2, name="full3", activation=None, trainable=trainable)
    
#     softmax = tf.nn.softmax(full3, name="softmax")
    
    
#     full1 = _create_full(conv3, 256, name="full1", activation=tf.nn.relu, trainable=trainable)
#     full2 = _create_full(full1, 128, name="full2", activation=tf.nn.relu, trainable=trainable)
#     full3 = _create_full(full2, 2, name="full3", activation=None, trainable=trainable)
    
#     softmax = tf.nn.softmax(full3, name="softmax")
    
    return softmax

In [7]:
def build_loss(logits, labels):
    with tf.name_scope("cross_entropy"):
        labels = tf.cast(labels, tf.int64)
        dense_labels = dense_to_one_hot(labels)
        clipped_logits = tf.clip_by_value(logits, 1e-10, 100.0)
        cross_entropy = -dense_labels * tf.log(clipped_logits)
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')

    return cross_entropy_mean

In [8]:
def build_train(loss_op, step, init_rate=0.01, decay_steps=6000):

    with tf.name_scope("train"):
        learning_rate = tf.train.exponential_decay(init_rate, step, decay_steps, 0.1, staircase=True)
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        optimizer = tf.train.AdamOptimizer(init_rate)
        grads = optimizer.compute_gradients(loss_op)

        apply_gradient_op = optimizer.apply_gradients(grads, global_step=step)

        for var in tf.trainable_variables():
            tf.histogram_summary(var.op.name, var)

        for grad, var in grads:
            if grad:
                tf.histogram_summary(var.op.name + '/gradients', grad)

    return apply_gradient_op

In [9]:
def load(saver, sess):
    saver.restore(sess, save_file_path)
    my_print("Model restored.\n")
    
def save(saver, sess):
    save_path = saver.save(sess, save_file_path)
    my_print("Model saved in file: %s\n" % save_path)

def train_classifier(need_load, N=10000, init_rate=0.1):
    
    with tf.Graph().as_default() as g: 
        
        step = tf.Variable(0, trainable=False, name="step")
        tf.add_to_collection("main", step)
    
        images1_train, images2_train, labels_train, images1_test, images2_test, labels_test = get_data_batch()
        images1_ph = tf.placeholder(tf.float32, [BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3], "images1")
        images2_ph = tf.placeholder(tf.float32, [BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3], "images2")
        labels_ph = tf.placeholder(tf.int64, [BATCH_SIZE], "labels")
        answer_op = build_classifier(images1_ph, images2_ph, trainable=True)
        loss_op = build_loss(answer_op, labels_ph)
        tf.scalar_summary("loss", loss_op)
        train_op = build_train(loss_op, step, init_rate=init_rate)
        correct_prediction = tf.equal(tf.argmax(answer_op, 1), labels_ph)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        accuracy_summary = tf.scalar_summary("accuracy", accuracy)

        merged_summaries = tf.merge_all_summaries()

        init = tf.initialize_all_variables()

        sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
        saver = tf.train.Saver(var_list=tf.get_collection("main"))
        coord = tf.train.Coordinator()
        writer = tf.train.SummaryWriter(logs_dir, sess.graph_def, flush_secs=30)

        sess.run(init)
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:

            if need_load:
                load(saver, sess)

            my_print("Starting...\n")

            for i in xrange(0, N):
#                 my_print("iteration: %i\n" % i)
                if i % 11 == 0:
                    im1, im2, lab = sess.run([images1_test, images2_test, labels_test])
                    feed = {
                        images1_ph : im1,
                        images2_ph : im2,
                        labels_ph : lab
                    }
                    result = sess.run([merged_summaries, accuracy, step], feed_dict=feed)
                    summary_str = result[0]
                    acc = result[1]
                    st = result[2]
#                     writer.add_summary(summary_str, st)
                    print("Accuracy at step %s: %s" % (st, acc))
                else:
                    im1, im2, lab = sess.run([images1_train, images2_train, labels_train])
                    feed = {
                        images1_ph : im1,
                        images2_ph : im2,
                        labels_ph : lab
                    }
                    result = sess.run([train_op, merged_summaries, accuracy, step], feed_dict=feed)
                    summary_str = result[1]
                    acc = result[2]
                    st = result[3]
                    writer.add_summary(summary_str, st)
                    print("Accuracy at step %s: %s" % (st, acc))
                    

                if i % 100 == 0:
                    save(saver, sess)
        finally:
            coord.request_stop()
            coord.join(threads)


In [10]:
train_classifier(need_load=False, N=100000, init_rate=0.001)

Total: 19088 images, train: 17179
Starting...
Accuracy at step 0: 0.453125
Model saved in file: ./save_col/model.ckpt
Accuracy at step 1: 0.453125
Accuracy at step 2: 0.53125
Accuracy at step 3: 0.5625
Accuracy at step 4: 0.53125
Accuracy at step 5: 0.546875
Accuracy at step 6: 0.453125
Accuracy at step 7: 0.5625
Accuracy at step 8: 0.390625
Accuracy at step 9: 0.46875
Accuracy at step 10: 0.578125
Accuracy at step 10: 0.578125
Accuracy at step 11: 0.515625
Accuracy at step 12: 0.53125
Accuracy at step 13: 0.53125
Accuracy at step 14: 0.4375
Accuracy at step 15: 0.5625
Accuracy at step 16: 0.46875
Accuracy at step 17: 0.4375
Accuracy at step 18: 0.40625
Accuracy at step 19: 0.5
Accuracy at step 20: 0.484375
Accuracy at step 20: 0.515625
Accuracy at step 21: 0.65625
Accuracy at step 22: 0.53125
Accuracy at step 23: 0.484375
Accuracy at step 24: 0.578125
Accuracy at step 25: 0.515625
Accuracy at step 26: 0.390625
Accuracy at step 27: 0.53125
Accuracy at step 28: 0.515625
Accuracy at step

KeyboardInterrupt: 